In [663]:
import pandas as pd
import numpy as np
lead_log = pd.read_csv("DE Dataset/lead_log.csv")
paid_trx = pd.read_csv("DE Dataset/paid_transactions.csv")
ref_reward = pd.read_csv("DE Dataset/referral_rewards.csv")
user_logs = pd.read_csv("DE Dataset/user_logs.csv")
user_ref_logs = pd.read_csv("DE Dataset/user_referral_logs.csv") #referral details
user_ref_stat = pd.read_csv("DE Dataset/user_referral_statuses.csv")
user_ref = pd.read_csv("DE Dataset/user_referrals.csv")


In [448]:
#1.0 -> 1
user_ref['referral_reward_id'] = user_ref['referral_reward_id'].apply(lambda x: str(int(float(x))) if pd.notna(x) else x)
user_ref = user_ref.astype({
    'referral_id': 'string',
    'referee_id': 'string',
    'referee_name': 'string',
    'referee_phone': 'string',
    'referral_reward_id' : 'string',
    'referral_source': 'string',
    'referrer_id': 'string',
    'transaction_id': 'string',
    'user_referral_status_id':'string'
})

user_ref_logs = user_ref_logs.astype({
    'user_referral_id': 'string',
    'source_transaction_id': 'string'
})

ref_reward = ref_reward.astype(
    {
      'id' : 'string'  
    }
)
user_ref_stat = user_ref_stat.astype(
    {
        'id' : 'string'
    }
)

paid_trx = paid_trx.astype(
    {
        'transaction_id':'string'
    }
)

In [449]:
user_ref_logs
# 9331c8f144dad5a3b8e4a10467b4343a
user_ref_logs.loc[user_ref_logs['user_referral_id'] == '7a03e01e0c17b2322845242a796adb5e']
valid_user_ref_logs = user_ref_logs.drop_duplicates(subset='user_referral_id')

In [664]:
user_ref_join_user_ref_logs = pd.merge(user_ref, valid_user_ref_logs, left_on=['referral_id'], right_on=['user_referral_id'], how='left')
user_ref_join_user_ref_logs = user_ref_join_user_ref_logs.rename(columns={'created_at': 'reward_granted_at'})

In [665]:
#get valid user data
valid_user_logs = user_logs.loc[user_logs['is_deleted'] == False]
valid_user_logs = valid_user_logs.drop_duplicates(subset='user_id')
user_logs.shape, valid_user_logs.shape
# valid_user_logs


((29, 8), (8, 8))

In [666]:
#get the referrer data
user_ref_join_user_ref_logs_with_referrer = pd.merge(user_ref_join_user_ref_logs, valid_user_logs, left_on=['referrer_id'], right_on=['user_id'], how='left')
user_ref_join_user_ref_logs_with_referrer = user_ref_join_user_ref_logs_with_referrer.drop(columns=['id_x', 'id_y', 'is_deleted'])
user_ref_join_user_ref_logs_with_referrer = user_ref_join_user_ref_logs_with_referrer.rename(columns={
    'name': 'referrer_name',
    'phone_number': 'referrer_phone_number',
    'homeclub': 'referrer_homeclub',
    'timezone_homeclub': 'referrertimezone_homeclub',
    'membership_expired_date': 'referrer_membership_expired_date'
})

user_ref_join_user_ref_logs_with_referrer.columns



Index(['referral_at', 'referral_id', 'referee_id', 'referee_name',
       'referee_phone', 'referral_reward_id', 'referral_source', 'referrer_id',
       'transaction_id', 'updated_at', 'user_referral_status_id',
       'user_referral_id', 'source_transaction_id', 'reward_granted_at',
       'is_reward_granted', 'user_id', 'referrer_name',
       'referrer_phone_number', 'referrer_homeclub',
       'referrertimezone_homeclub', 'referrer_membership_expired_date'],
      dtype='object')

In [679]:
#join paid_transaction
main_table = pd.merge(user_ref_join_user_ref_logs_with_referrer, paid_trx, on=['transaction_id'], how='left')
# main_table = main_table.rename(columns={'created_at':'user_referral_created_at'})
main_table.shape


(46, 26)

In [680]:
#join reward
main_table_1 = pd.merge(main_table, ref_reward, left_on=['referral_reward_id'], right_on=['id'], how='left')
main_table_1 = main_table_1.rename(columns={'created_at':'ref_rewards_created_at'})
main_table_1.shape


(46, 30)

In [681]:
#join user_ref_status
main_table_2 = pd.merge(main_table_1, user_ref_stat, left_on=['user_referral_status_id'], right_on=['id'], how='left')
main_table_2 = main_table_2.rename(columns={'created_at':'user_referral__statuses_created_at'})

main_table_2.shape


(46, 33)

In [670]:
#join lead_logs if source lead, otherwise using name/phone
lead_condition = main_table_2.loc[main_table_2['referral_source'] == 'Lead']
non_lead_condition = main_table_2.loc[main_table_2['referral_source'] != 'Lead']
non_lead_condition.shape, lead_condition.shape
# lead_condition.columns


((35, 33), (11, 33))

In [671]:
valid_lead_log = lead_log.drop_duplicates(subset='lead_id')
valid_lead_log


,id,lead_id,source_category,created_at,preferred_location,timezone_location,current_status
0,1,774656feb3119be1d8d6d893429c7cc0,Online,2024-05-13T06:38:58.322Z,GAJAH MADA,Asia/Jakarta,Fresh
1,2,bbd3c7b2448041dc5339b23878cacd79,Online,2024-05-10T04:16:27.919Z,ADITYAWARMAN,Asia/Jakarta,Warm
2,3,a90dcb9c1fc9c63f1e2dc7dbd79ecfd2,Offline,2024-01-31T08:53:22.403Z,BENHIL,Asia/Jakarta,Deal
3,4,c50f607bbae83f7c53551476e11997eb,Online,2024-05-13T06:31:39.597Z,BENHIL,Asia/Jakarta,Deal
4,5,08b8ab0371b6a0cb79319628505c446c,Online,2024-05-01T02:54:03.482Z,BENHIL,Asia/Jakarta,Fresh


In [672]:
#lead join lead
main_table_lead = pd.merge(lead_condition, valid_lead_log, left_on=['referee_id'], right_on=['lead_id'], how='left')
main_table_lead = main_table_lead.drop(['current_status', 'timezone_location', 'preferred_location', 'created_at', 'lead_id', 'id'], axis=1)
main_table_lead

,referral_at,referral_id,referee_id,referee_name,referee_phone,referral_reward_id,referral_source,referrer_id,transaction_id,updated_at,...,timezone_transaction,transaction_type,id_x,reward_value,ref_rewards_created_at,reward_type,id_y,description,user_referral__statuses_created_at,source_category
0,2024-05-22T09:03:10Z,b6732d2ca0bc7acbc1b39cb8a04a96cd,8cba435bca358b92462a30bc999598bf,2c44e14b1d36c87097edaf78e3110392,41440157b33b9eff6c7054134a613fc0,NaN,Lead,4acfa96cb521b6963b50bcef95bee485,NaN,2024-05-22T09:03:10Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,NaN
1,2024-03-20T07:48:49Z,d452feec722a5cabc03645bb089e2735,08b8ab0371b6a0cb79319628505c446c,360701f01f783634925ce89e59decc37,ce52ad7070f305b43a784b8503dbde13,1.0,Lead,cbac8ed440d53d1c6c187874c4193db2,7709d17eaec03665f58904b72c8ab444,2024-05-01T02:58:57Z,...,NaN,NaN,1.0,10 days,2024-03-13T02:44:32Z,1.0,1,Menunggu,2024-03-08T08:49:37Z,Online
2,2024-05-22T08:47:24Z,7bf019d6eec0fc5f6485fb4d1968fe80,b64ce1531c508d3090a76ae678647f20,c315681adbaee59c3c0e945ed9436173,7f2786e2875779cc5279bf6d834c862c,NaN,Lead,4acfa96cb521b6963b50bcef95bee485,NaN,2024-05-22T08:47:24Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,NaN
3,2024-05-10T04:16:29Z,2d07be15757d1ea6d85b96f4d46c4a37,bbd3c7b2448041dc5339b23878cacd79,efd4849c38c28fe336382a1d244b9042,3b3d9a726e5dacd9315bbca8d754b8e5,NaN,Lead,cadba628952bde1670ac4c0792afc28a,NaN,2024-05-10T04:16:29Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,Online
4,2024-05-06T08:02:16Z,90abb7a3fab6efe0f5017b11df3c2ffc,a90dcb9c1fc9c63f1e2dc7dbd79ecfd2,d1281ff14d6b879cfdc35b35aa4e0e48,51249abf6bb88596d2a1757b1e258114,NaN,Lead,2c71c5d66c7e12a0b3c200ba6ed3b78e,NaN,2024-05-06T08:02:16Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,Offline
5,2024-05-13T06:38:59Z,251cb17f5a0a266d4eec7f3f478ad245,774656feb3119be1d8d6d893429c7cc0,4bbc7361bafa86ca236399f1029f537f,ca4712fd139cd74c83b5f4f43dc6cc4c,NaN,Lead,4acfa96cb521b6963b50bcef95bee485,NaN,2024-05-13T06:38:59Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,Online
6,2024-05-13T06:31:40Z,61717d106753428549731b63554d70a5,c50f607bbae83f7c53551476e11997eb,0a151f113d078794bae1b6b2d0e432c4,c72ff35974713260b8153d81c84b3fa5,1.0,Lead,4acfa96cb521b6963b50bcef95bee485,da0071e2bb7794ac19ff4dbbb6e772f0,2024-05-23T05:32:21Z,...,NaN,NaN,1.0,10 days,2024-03-13T02:44:32Z,1.0,3,Tidak Berhasil,2024-03-08T10:07:10Z,Online
7,2024-04-16T07:09:25Z,30a0cdbe9f518b9938bf9b9399f8b048,08b8ab0371b6a0cb79319628505c446c,360701f01f783634925ce89e59decc37,ce52ad7070f305b43a784b8503dbde13,NaN,Lead,b755269324c62c3a6b0bb6568f5d4d26,4c8cae052f19cea66544affc759b76ee,2024-05-01T02:58:57Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,Online
8,2024-03-20T07:49:17Z,8882f8f02e658bd06627f9e0aedf9325,08b8ab0371b6a0cb79319628505c446c,360701f01f783634925ce89e59decc37,ce52ad7070f305b43a784b8503dbde13,2.0,Lead,2c71c5d66c7e12a0b3c200ba6ed3b78e,NaN,2024-05-01T02:58:58Z,...,NaN,NaN,2.0,20 days,2024-03-13T02:45:11Z,1.0,1,Menunggu,2024-03-08T08:49:37Z,Online
9,2024-05-24T04:57:33Z,ca06593605d62bc27dacb93533278f44,04aec3adea13099cb24c4cc784f05f3d,f253ae67d684e206819334deb74d4c38,bb7e63d99dd0592baa702b4cb5713187,NaN,Lead,1eab0611433407d851896635a5b735f5,NaN,2024-05-24T04:57:33Z,...,NaN,NaN,NaN,NaN,NaN,NaN,1,Menunggu,2024-03-08T08:49:37Z,NaN


In [673]:
non_lead_condition.referral_source
non_lead_condition.loc[non_lead_condition.referral_source == 'Draft Transaction', 'source_category'] = 'Offline'
non_lead_condition.loc[non_lead_condition.referral_source == 'User Sign Up', 'source_category'] = 'Online'
non_lead_condition.shape


/var/folders/pt/js2dqlw53qg1snpf579gfc0r0000gn/T/ipykernel_898/3446176607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_lead_condition.loc[non_lead_condition.referral_source == 'Draft Transaction', 'source_category'] = 'Offline'


(35, 34)

In [682]:
main_table_3 = pd.concat([lead_condition, non_lead_condition], ignore_index=True)
main_table_3.shape

(46, 34)

In [687]:
final_table = main_table_3[[
                            #referral_details_id
                            'referral_id'
                            , 'referral_source'
                            , 'source_category' # referral_source_category
                            , 'referral_at'
                            , 'referrer_id'
                            , 'referrer_name' # 'referrer_name'
                            , 'referrer_phone_number'
                            , 'referrer_homeclub'
                            , 'referrer_membership_expired_date' # hapus
                            , 'referee_id'
                            , 'referee_name'
                            , 'referee_phone'
                            , 'description' # referral_status
                            , 'reward_value' # num_reward_days
                            , 'transaction_id'
                            , 'transaction_status'
                            , 'transaction_at'
                            , 'transaction_location'
                            , 'transaction_type'
                            , 'updated_at'
                            , 'reward_granted_at'
                            , 'is_reward_granted' # hapus
                            , 'ref_rewards_created_at' #hapus
                            , 'user_referral__statuses_created_at' #hapus
                            #is_business_logic_valid
                            ]]

# Add an auto-incrementing 'ID' column with leading zeros
final_table['referral_details_id'] = [f'{i:05}' for i in range(1, len(final_table) + 1)]
final_table['is_business_logic_valid'] = ''
final_table = final_table.rename(columns={
    'source_category' : 'referral_source_category'
    , 'description' : 'referral_status'
    , 'reward_value' : 'num_reward_days'
})

final_table['num_reward_days'] = final_table['num_reward_days'].str.replace(' days', '', regex=False)
final_table['num_reward_days'] = pd.to_numeric(final_table['num_reward_days'], errors='coerce')

final_table['is_reward_granted'] = final_table['is_reward_granted'].fillna(False)
final_table['is_reward_granted'] = final_table['is_reward_granted'].astype('bool')

final_table['transaction_at'] = pd.to_datetime(final_table['transaction_at'])
final_table['transaction_at'] = final_table['transaction_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
final_table['transaction_at'] = pd.to_datetime(final_table['transaction_at'], format='%Y-%m-%d %H:%M:%S')


final_table['referral_at'] = pd.to_datetime(final_table['referral_at'])
final_table['referral_at'] = final_table['referral_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
final_table['referral_at'] = pd.to_datetime(final_table['referral_at'], format='%Y-%m-%d %H:%M:%S')

final_table['user_referral__statuses_created_at'] = pd.to_datetime(final_table['user_referral__statuses_created_at'])
final_table['user_referral__statuses_created_at'] = final_table['user_referral__statuses_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
final_table['user_referral__statuses_created_at'] = pd.to_datetime(final_table['user_referral__statuses_created_at'], format='%Y-%m-%d %H:%M:%S')

final_table['ref_rewards_created_at'] = pd.to_datetime(final_table['ref_rewards_created_at'])
final_table['ref_rewards_created_at'] = final_table['ref_rewards_created_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
final_table['ref_rewards_created_at'] = pd.to_datetime(final_table['ref_rewards_created_at'], format='%Y-%m-%d %H:%M:%S')

final_table['referrer_membership_expired_date'] = pd.to_datetime(final_table['referrer_membership_expired_date'])

cols = ['referral_details_id'] + [col for col in final_table.columns if col != 'referral_details_id']
final_table = final_table[cols]

def fill_na_based_on_dtype(series):
    if series.dtype == 'float64' or series.dtype == 'int64':
        return series.fillna(0)
    elif series.dtype == 'object':
        return series.fillna('')
    elif series.dtype == 'string':
        return series.fillna('')
    elif pd.api.types.is_datetime64_any_dtype(series):
        return series.fillna(pd.NaT)
    else:
        return series

final_table = final_table.apply(fill_na_based_on_dtype)


# Menghitung jumlah nilai null pada setiap kolom
null_counts = final_table.isnull().sum()
print("Jumlah nilai null per kolom:")
print(null_counts)

final_table.shape


Jumlah nilai null per kolom:
referral_details_id                    0
referral_id                            0
referral_source                        0
referral_source_category               0
referral_at                            0
referrer_id                            0
referrer_name                          0
referrer_phone_number                  0
referrer_homeclub                      0
referrer_membership_expired_date      18
referee_id                             0
referee_name                           0
referee_phone                          0
referral_status                        0
num_reward_days                        0
transaction_id                         0
transaction_status                     0
transaction_at                        32
transaction_location                   0
transaction_type                       0
updated_at                             0
reward_granted_at                      0
is_reward_granted                      0
ref_rewards_created_at      

/var/folders/pt/js2dqlw53qg1snpf579gfc0r0000gn/T/ipykernel_898/2161704700.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['referral_details_id'] = [f'{i:05}' for i in range(1, len(final_table) + 1)]
/var/folders/pt/js2dqlw53qg1snpf579gfc0r0000gn/T/ipykernel_898/2161704700.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['is_business_logic_valid'] = ''


(46, 26)

In [676]:
import pandas as pd

# Get current datetime
now = pd.Timestamp.now().strftime('%Y-%m-%d')

now, (type(final_table['referrer_membership_expired_date']))



('2024-08-17', pandas.core.series.Series)

In [689]:
final_table['is_business_logic_valid'] = False
final_table['is_business_logic_valid'] = final_table['is_business_logic_valid'].astype(bool)
final_table.loc[
                (final_table['num_reward_days'] > 0) 
                & (final_table['referral_status'] == 'Berhasil') 
                & (final_table['transaction_id'].notna().any()) 
                & (final_table['transaction_status'] == 'PAID') 
                & (final_table['transaction_type'] == 'NEW') 
                & (final_table['transaction_at'] >= final_table['referral_at']) 
                & (final_table['transaction_at'].dt.month == final_table['referral_at'].dt.month)
                & (final_table['referrer_membership_expired_date'] >= pd.Timestamp(now))
                & (final_table['is_reward_granted'] == True)
                , 'is_business_logic_valid'] = True


final_table.loc[
                (final_table['referral_status'].isin(['Menunggu', 'Tidak Berhasil']))
                & (final_table['num_reward_days'] == 0) 
                , 'is_business_logic_valid'] = True

final_table.loc[
                (final_table['num_reward_days'] > 0) 
                & (final_table['referral_status'] != 'Berhasil')
                , 'is_business_logic_valid'] = False

final_table.loc[
                (final_table['num_reward_days'] > 0) 
                & (final_table['transaction_id'].isnull())
                , 'is_business_logic_valid'] = False

final_table.loc[
                (final_table['num_reward_days'] == 0) 
                & (final_table['transaction_id'].notna().any())
                & (final_table['transaction_status'] == 'PAID')
                & (final_table['transaction_at'] > final_table['referral_at']) 
                , 'is_business_logic_valid'] = False


final_table.loc[
                (final_table['referral_status'] == 'Berhasil')
                & (final_table['num_reward_days'] == 0)
                , 'is_business_logic_valid'] = False

final_table.loc[
                (final_table['transaction_at'] < final_table['referral_at']) 
                , 'is_business_logic_valid'] = False

#adding logic
final_table.loc[
                (final_table['ref_rewards_created_at'] >= final_table['user_referral__statuses_created_at']) 
                , 'is_business_logic_valid'] = False


final_table[['user_referral__statuses_created_at', 'ref_rewards_created_at', 'referral_source', 'num_reward_days', 'referral_status', 'transaction_id', 'transaction_status', 'transaction_type', 'transaction_at', 'referral_at', 'referrer_membership_expired_date', 'is_reward_granted', 'is_business_logic_valid']]
final_table.is_business_logic_valid.value_counts()

is_business_logic_valid
True     32
False    14
Name: count, dtype: int64

In [639]:
a = final_table.loc[final_table.is_business_logic_valid == '']
a.to_csv("DE Dataset/null_business_logic.csv")


In [638]:
b = user_ref_logs.loc[user_ref_logs.is_reward_granted == True]
c = user_ref_logs.loc[user_ref_logs.user_referral_id == '7a03e01e0c17b2322845242a796adb5e']
c

,id,user_referral_id,source_transaction_id,created_at,is_reward_granted
2,3,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-29T10:25:56Z,False
5,6,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-22T15:22:55Z,False
7,8,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-28T08:48:13Z,False
16,17,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-29T04:03:18Z,False
23,24,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-28T10:27:03Z,False
27,28,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-28T08:42:00Z,False
31,32,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-29T10:23:38Z,False
43,44,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-29T10:00:32Z,False
49,50,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-28T08:46:53Z,False
55,56,7a03e01e0c17b2322845242a796adb5e,NaN,2024-04-29T10:27:02Z,False


In [647]:
z = main_table_3.loc[main_table_3['referral_id'].isin(['9e9324e6fde29bb0d230654b38ccfdd4', '835e990334fe52dec832043f19ddfd07', 'a95d6a6cefee850333bb2df257cb2f79'])]
z.to_csv("DE Dataset/null_business_logic_3.csv")